검색어 구체화 및 다중 검색: 하나의 포괄적인 검색어 대신, 여러 개의 구체적인 검색어를 리스트로 만들어 순차적으로 크롤링합니다. 이를 통해 더 다양하고 관련성 높은 이미지를 수집할 확률을 높입니다.
이미지 URL 필터링: Pixabay의 이미지 URL 구조를 분석하여, 고화질 원본 이미지일 가능성이 높은 특정 패턴을 가진 URL만 수집하도록 필터링 로직을 추가합니다.

In [3]:
import time
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import zipfile

def image_crawler(keywords, num_images_per_keyword, save_folder):
    """
    Selenium을 이용해 Pixabay에서 여러 구체적인 키워드로 이미지를 크롤링하고 저장하는 함수
    :param keywords: 검색할 키워드 리스트 (예: ['red apple', 'green apple'])
    :param num_images_per_keyword: 각 키워드당 다운로드할 이미지 개수
    :param save_folder: 이미지를 저장할 폴더 경로
    """
    # 1. 저장할 폴더 생성
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
        print(f"폴더 생성 완료: {save_folder}")

    # 2. 웹 드라이버 실행
    driver = webdriver.Chrome()

    collected_urls = set() # 중복된 URL 수집을 방지하기 위한 set

    # 3. 각 키워드별로 크롤링 반복
    for keyword in keywords:
        print("\n" + "="*30)
        print(f"'{keyword}' 키워드로 검색을 시작합니다...")

        # Pixabay 검색 URL로 접속
        url = f"https://pixabay.com/images/search/{keyword.replace(' ', '%20')}/"
        driver.get(url)
        driver.implicitly_wait(10)
        time.sleep(2)

        # 4. 스크롤 다운하여 이미지 동적 로딩
        last_height = driver.execute_script("return document.body.scrollHeight")
        required_images = len(collected_urls) + num_images_per_keyword

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)
            new_height = driver.execute_script("return document.body.scrollHeight")

            # 현재 로드된 이미지 개수 확인
            # Pixabay의 이미지 컨테이너 클래스는 'container--M39oN'의 일부인 'media_container'를 포함하는 경우가 많음.
            # 더 안정적으로 <img> 태그를 직접 찾습니다.
            images = driver.find_elements(By.TAG_NAME, 'img')

            if len(images) >= required_images or new_height == last_height:
                break
            last_height = new_height

        # 5. 이미지 URL 수집 및 필터링
        images = driver.find_elements(By.TAG_NAME, 'img')
        urls_for_keyword = 0
        for image in images:
            src = image.get_attribute('src')
            # ⭐ 전략 2: URL 필터링 적용 ⭐
            # Pixabay의 고화질 이미지 CDN URL은 보통 'cdn.pixabay.com/photo/' 패턴을 포함합니다.
            # 또한, 너무 작은 썸네일 이미지(base64 인코딩된 이미지)를 제외합니다.
            if src and src.startswith('https://cdn.pixabay.com/photo/'):
                if src not in collected_urls:
                    collected_urls.add(src)
                    urls_for_keyword += 1
                    if urls_for_keyword >= num_images_per_keyword:
                        break # 현재 키워드에 대한 목표 수량 달성

    print("\n" + "="*50)
    print(f"총 {len(collected_urls)}개의 유니크한 이미지 URL을 수집했습니다.")
    driver.quit()

    # 6. 수집된 URL로부터 이미지 다운로드
    print("이미지 다운로드를 시작합니다...")
    for i, url in enumerate(list(collected_urls)):
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                # 파일명 설정 (URL의 마지막 부분을 사용하거나 간단하게 숫자로)
                filename = os.path.join(save_folder, f"{save_folder.split('/')[-1]}_{i+1}.jpg")
                with open(filename, 'wb') as f:
                    f.write(response.content)
                print(f"[{i+1}/{len(collected_urls)}] 이미지 저장 완료: {filename}")
            else:
                print(f"이미지 다운로드 실패 (상태 코드: {response.status_code}): {url}")
            time.sleep(0.5)
        except Exception as e:
            print(f"이미지 다운로드 중 오류 발생: {e}")

def zip_directory(folder_path, output_path):
    # (이전과 동일한 zip_directory 함수 내용)
    print(f"'{folder_path}' 폴더를 압축하는 중...")
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, os.path.dirname(folder_path))
                zipf.write(file_path, arcname=arcname)
    print(f"압축 완료: {output_path}")

# --- 크롤러 실행 ---
if __name__ == "__main__":

    # ⭐ 전략 1: 구체적인 다중 키워드 사용 ⭐
    apple_keywords = ['red apple fruit', 'green apple fruit', 'apple on tree', 'apple isolated']
    orange_keywords = ['orange fruit isolated', 'orange fruit on tree', 'sliced orange fruit']

    # 각 키워드당 30장씩 수집하여 총 120장(사과), 90장(오렌지) 정도를 목표로 함
    num_target = 100 # 최종 목표 이미지 개수

    # 1. 사과 이미지 수집
    apple_save_folder = './images/apple'
    num_per_apple_keyword = num_target // len(apple_keywords) + 1
    image_crawler(keywords=apple_keywords, num_images_per_keyword=num_per_apple_keyword, save_folder=apple_save_folder)

    apple_zip_path = './images/apple.zip'
    zip_directory(folder_path=apple_save_folder, output_path=apple_zip_path)

    print("\n" + "="*50 + "\n")

    # 2. 오렌지 이미지 수집
    orange_save_folder = './images/orange'
    num_per_orange_keyword = num_target // len(orange_keywords) + 1
    image_crawler(keywords=orange_keywords, num_images_per_keyword=num_per_orange_keyword, save_folder=orange_save_folder)

    orange_zip_path = './images/orange.zip'
    zip_directory(folder_path=orange_save_folder, output_path=orange_zip_path)

    print("\n모든 이미지 수집 및 압축이 완료되었습니다.")


'red apple fruit' 키워드로 검색을 시작합니다...

'green apple fruit' 키워드로 검색을 시작합니다...

'apple on tree' 키워드로 검색을 시작합니다...

'apple isolated' 키워드로 검색을 시작합니다...

총 78개의 유니크한 이미지 URL을 수집했습니다.
이미지 다운로드를 시작합니다...
[1/78] 이미지 저장 완료: ./images/apple\apple_1.jpg
[2/78] 이미지 저장 완료: ./images/apple\apple_2.jpg
[3/78] 이미지 저장 완료: ./images/apple\apple_3.jpg
[4/78] 이미지 저장 완료: ./images/apple\apple_4.jpg
[5/78] 이미지 저장 완료: ./images/apple\apple_5.jpg
[6/78] 이미지 저장 완료: ./images/apple\apple_6.jpg
[7/78] 이미지 저장 완료: ./images/apple\apple_7.jpg
[8/78] 이미지 저장 완료: ./images/apple\apple_8.jpg
[9/78] 이미지 저장 완료: ./images/apple\apple_9.jpg
[10/78] 이미지 저장 완료: ./images/apple\apple_10.jpg
[11/78] 이미지 저장 완료: ./images/apple\apple_11.jpg
[12/78] 이미지 저장 완료: ./images/apple\apple_12.jpg
[13/78] 이미지 저장 완료: ./images/apple\apple_13.jpg
[14/78] 이미지 저장 완료: ./images/apple\apple_14.jpg
[15/78] 이미지 저장 완료: ./images/apple\apple_15.jpg
[16/78] 이미지 저장 완료: ./images/apple\apple_16.jpg
[17/78] 이미지 저장 완료: ./images/apple\apple_17.jpg
[18/78] 이미지 저장 완료: ./im

image_crawler 함수 파라미터 변경:
keyword (문자열) -> keywords (리스트): 여러 개의 검색어를 받을 수 있도록 변경했습니다.
num_images_to_download -> num_images_per_keyword: 각 키워드당 몇 개의 이미지를 목표로 할지 지정하도록 변경했습니다.
다중 키워드 루프:
for keyword in keywords: 루프를 추가하여, 전달받은 키워드 리스트를 순회하며 각 키워드에 대해 검색과 스크롤, URL 수집을 반복합니다.
URL 중복 제거:
collected_urls = set(): 수집한 URL을 저장하기 위해 리스트 대신 **set**을 사용합니다. set은 중복된 값을 허용하지 않으므로, 다른 키워드 검색 결과에서 동일한 이미지 URL이 수집되더라도 자동으로 중복이 제거됩니다.
⭐ URL 필터링 로직 추가 (핵심 개선 사항) ⭐
if src and src.startswith('https://cdn.pixabay.com/photo/'):: <img> 태그의 src 속성을 가져온 뒤, 해당 URL이 Pixabay의 고화질 사진 CDN 서버 주소로 시작하는지 확인합니다.
이 필터를 통해 광고, 아이콘, base64로 인코딩된 작은 썸네일 이미지 등 관련 없는 URL들을 효과적으로 걸러낼 수 있습니다. 이 부분이 데이터 순도를 높이는 데 가장 큰 역할을 합니다.
실행 부분 (if __name__ == "__main__":) 수정:
apple_keywords, orange_keywords 와 같이 구체적인 검색어 리스트를 정의했습니다. 이 리스트를 수정하여 검색 결과를 제어할 수 있습니다.
각 키워드당 다운로드할 이미지 개수를 num_target // len(keywords) + 1 로 자동 계산하여, 전체적으로 목표하는 이미지 개수(100장)와 비슷하게 수집하도록 설정했습니다.